# Import library

In [40]:
import pandas as pd
import yaml

# Variable config

## Read yaml file 

In [41]:
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [42]:
data_path = config['DATA_PATH'] ## '../data/dress_rental_prices news.csv'

# Read data

In [43]:
df = pd.read_csv(data_path)

In [44]:
df.head()

,numbers,ID,Name,Brand,Colour,Catagories,Price
0,0,74416,Runway stripe dress,Stella McCartney,beige,dresses,111
1,1,73815,Reformation Kourtney Dress,Reformation,beige,dresses,50
2,2,73801,Ivory Viola bridal dress,Ghost,beige,dresses,75
3,3,73718,Pasu Dress - Rhino Tusk,Coucoo,beige,dresses,37
4,4,73605,Ellen,RIXO,beige,dresses,47


# Explore data

## data type

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29997 entries, 0 to 29996
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   numbers     29997 non-null  int64 
 1   ID          29997 non-null  int64 
 2   Name        29995 non-null  object
 3   Brand       29997 non-null  object
 4   Colour      29997 non-null  object
 5   Catagories  29997 non-null  object
 6   Price       29997 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.6+ MB


## describe

In [46]:
df.describe(percentiles=[.1, .90, .95, .99])

,numbers,ID,Price
count,29997.000000,29997.000000,29997.000000
mean,14998.000000,42548.336400,61.668800
std,8659.532349,19867.282746,38.138688
min,0.000000,122.000000,11.000000
10%,2999.600000,14777.600000,32.000000
50%,14998.000000,42902.000000,51.000000
90%,26996.400000,68848.400000,102.000000
95%,28496.200000,71589.200000,139.000000
99%,29696.040000,73993.120000,195.000000
max,29996.000000,75024.000000,793.000000


## ydata-profiling

In [47]:
## let see in exlore_data.ipynb

## nan value

In [48]:
df.isna().sum()

numbers       0
ID            0
Name          2
Brand         0
Colour        0
Catagories    0
Price         0
dtype: int64

# Transform data

## drop nan value

In [49]:
df = df[~df['Name'].isna()]

## drop numbers column

In [50]:
df.drop('numbers', axis= 1 , inplace = True)

## change data type

In [51]:
df['ID'] = df['ID'].astype('object')
df['Brand'] = df['Brand'].astype('category')
df['Colour'] = df['Colour'].astype('category')

## split categories and replace nan with no-sub-category

In [52]:
sub_categories = df['Catagories'].str.split(',', expand=True)
sub_categories = sub_categories.fillna('no-sub-categories')
sub_categories.columns = [f'sub-Catagories-{i+1}' for i in range(sub_categories.shape[1])]
for sub in sub_categories.columns:
    sub_categories[sub] = sub_categories[sub].astype('category')
df = pd.concat([df, sub_categories], axis=1)
df = df.drop(columns=['Catagories'])

## filter data with less than 400

In [53]:
df = df[df['Price'] < 400]

# Feature engineering

## Calculate average price by brand and add as a new column

In [54]:
df['Avg_Price_By_Brand'] = df.groupby('Brand')['Price'].transform('mean')

C:\Users\Rattapon.San\AppData\Local\Temp\ipykernel_19036\1663022941.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Avg_Price_By_Brand'] = df.groupby('Brand')['Price'].transform('mean')


## Calculate average price by (brand, colour) and add as a new column

In [55]:
df['Avg_Price_By_Brand_Colour'] = df.groupby(['Brand', 'Colour'])['Price'].transform('mean')

C:\Users\Rattapon.San\AppData\Local\Temp\ipykernel_19036\3750462187.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['Avg_Price_By_Brand_Colour'] = df.groupby(['Brand', 'Colour'])['Price'].transform('mean')


## Create a new column to count the number of sub-categories that are not 'no-sub-categories'

In [56]:
df['Category_Count'] = (sub_categories != 'no-sub-categories').sum(axis=1)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29974 entries, 0 to 29996
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   ID                         29974 non-null  object  
 1   Name                       29974 non-null  object  
 2   Brand                      29974 non-null  category
 3   Colour                     29974 non-null  category
 4   Price                      29974 non-null  int64   
 5   sub-Catagories-1           29974 non-null  category
 6   sub-Catagories-2           29974 non-null  category
 7   sub-Catagories-3           29974 non-null  category
 8   sub-Catagories-4           29974 non-null  category
 9   sub-Catagories-5           29974 non-null  category
 10  sub-Catagories-6           29974 non-null  category
 11  sub-Catagories-7           29974 non-null  category
 12  sub-Catagories-8           29974 non-null  category
 13  sub-Catagories-9           29974 non

# Split data to train, validation, test

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
def split_dataframe(df, train_size=0.7, validation_size=0.2, test_size=0.1, stratify=None):
    """
    Splits a DataFrame into train, validation, and test sets.
    
    Parameters:
    df (DataFrame): The input DataFrame to be split.
    train_size (float): Proportion of the dataset to include in the train split (0 to 1).
    validation_size (float): Proportion of the dataset to include in the validation split (0 to 1).
    test_size (float): Proportion of the dataset to include in the test split (0 to 1).
    stratify (str or None): Column to be used for stratification. Default is None.
    
    Returns:
    train_df (DataFrame): Training set.
    validation_df (DataFrame): Validation set.
    test_df (DataFrame): Test set.

    Example:
    >>> import pandas as pd
    >>> data = {'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                'Feature': [0.1, 0.2, 0.2, 0.4, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                'Target': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]}
    >>> df = pd.DataFrame(data)
    >>> train_df, validation_df, test_df = split_dataframe(df, stratify='Target')
    >>> print(train_df)
    >>> print(validation_df)
    >>> print(test_df)
    """
    
    # Stratify parameter for the split (can be None)
    stratify_param = None
    if stratify is not None:
        stratify_param = df[stratify]
    
    # Split into train+validation and test sets
    train_validation_df, test_df = train_test_split(df, test_size=test_size, random_state=42, stratify=stratify_param)
    
    if validation_size == 0:
        return train_validation_df, None, test_df
    else:
        # Adjust validation size to account for the test set already being removed
        adjusted_validation_size = validation_size / (1 - test_size)
        
        # Stratify parameter for splitting train into train and validation
        stratify_train_param = None
        if stratify is not None:
            stratify_train_param = train_validation_df[stratify]
        
        # Split the remaining data into train and validation sets
        train_df, validation_df = train_test_split(train_validation_df, test_size=adjusted_validation_size, random_state=42, stratify=stratify_train_param)
        
        return train_df, validation_df, test_df

In [60]:
train, validation, test = split_dataframe(df)